In [4]:
import pandas as pd
from pathlib import Path
from ast import literal_eval

In [23]:
df = pd.read_csv('/media/ts/SSD_ubuntu/datasets/DiDeMo/annot_clip_train.csv')
df

,filename,captions
0,train/100233434@N08_9622921646_c39ac10ea3_clip...,"['wind moves the leafs.', 'we first start to l..."
1,train/100233434@N08_9622921646_c39ac10ea3_clip...,"['during these three segments, the camera stop..."
2,train/100233434@N08_9815525316_b919341a87_clip...,['full square that plant is in comes into view']
3,train/100233434@N08_9815525316_b919341a87_clip...,"['flowerpots can be seen for the first time', ..."
4,train/100233434@N08_9815525316_b919341a87_clip...,['no camera movement']
...,...,...
21503,train/99917955@N00_5522494227_ce46cdc952_clip_...,['the wall of rocks exits view the first time']
21504,train/99996214@N00_5277035407_bb8738f945_clip_...,['person is petting dog in the back']
21505,train/99996214@N00_5277035407_bb8738f945_clip_...,['White dog walks from right side of other dog...
21506,train/99996214@N00_5277035407_bb8738f945_clip_...,"[""a white dog wearing a pink cast on it's leg ..."


In [24]:
data_root = Path("/media/ts/SSD_ubuntu/datasets/DiDeMo/video_clips/")

for i, row in df.iterrows():
    p = Path(row['filename'])
    if '.' in p.stem:
        row = row.copy()
        new_p = p.parent / (p.stem.replace('.', '') + p.suffix)
        df.loc[i, 'filename'] = new_p
        print(p, "=>", new_p)
        (data_root / p).rename(data_root / new_p)

train/10186213@N07_8721409789_15f7bde569._clip_0_5.mp4 => train/10186213@N07_8721409789_15f7bde569_clip_0_5.mp4
train/10186213@N07_8721409789_15f7bde569._clip_10_15.mp4 => train/10186213@N07_8721409789_15f7bde569_clip_10_15.mp4
train/10186213@N07_8721409789_15f7bde569._clip_10_25.mp4 => train/10186213@N07_8721409789_15f7bde569_clip_10_25.mp4
train/10186213@N07_8721409789_15f7bde569._clip_20_25.mp4 => train/10186213@N07_8721409789_15f7bde569_clip_20_25.mp4
train/10383932@N04_7163092715_a7b9931e68._clip_0_5.mp4 => train/10383932@N04_7163092715_a7b9931e68_clip_0_5.mp4
train/10383932@N04_7163092715_a7b9931e68._clip_10_15.mp4 => train/10383932@N04_7163092715_a7b9931e68_clip_10_15.mp4
train/10383932@N04_7163092715_a7b9931e68._clip_10_20.mp4 => train/10383932@N04_7163092715_a7b9931e68_clip_10_20.mp4
train/10383932@N04_7163092715_a7b9931e68._clip_15_20.mp4 => train/10383932@N04_7163092715_a7b9931e68_clip_15_20.mp4
train/10754072@N00_7363032598_b904b24fe4._clip_0_5.mp4 => train/10754072@N00_736

In [25]:
exists = df.filename.apply(lambda x: (data_root / x).exists())
exists.sum() == len(exists)

True

In [24]:
import re

def clean(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(' .', '.').strip()
    text = text if text.endswith('.') else text + '.'
    # Upper first letter
    text = text[0].upper() + text[1:]
    return text



for split in ['train', 'val', 'test']:
    df = pd.read_csv(f'/media/t9s9/SSD_ubuntu/datasets/DiDeMo/annot_clip_{split}.csv')
    df['captions'] = df['captions'].apply(lambda x: [clean(s) for s in literal_eval(x)])
    df['name'] = df.filename.apply(lambda x: Path(x).stem)

    df_audio = pd.read_parquet(f'/media/t9s9/SSD_ubuntu/datasets/DiDeMo/annot_auditory_{split}.parquet')
    df_audio['name'] = df_audio.filename.apply(lambda x: Path(x).stem)
    df_audio.rename(columns={'caption': 'captions'}, inplace=True)
    df_audio['captions'] = df_audio['captions'].apply(lambda x: [clean(x)])

    df_new = (
        df.merge(df_audio, on='name', how='inner', validate='1:1', suffixes=('_visual', '_auditory'))
          .drop(columns=['name', 'filename_visual'])
          .rename(columns={'filename_auditory': 'filename'})
          .reindex(columns=['filename', 'captions_visual', 'captions_auditory'])
              )
    df_new.to_parquet(f'/media/t9s9/SSD_ubuntu/datasets/DiDeMo/annot_full_{split}.parquet', index=False)

In [26]:
df_new.iloc[12]

filename             test/10218698@N06_2860871668_bd2ae9df3a_clip_5...
captions_visual      [Singer grabs microphone cord with his left ha...
captions_auditory                   [A song is being played by a man.]
Name: 12, dtype: object